In [1]:
import cv2
import numpy as np

def track_and_annotate_ball(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Video cannot be opened.")
        return

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    positions = []  # Track positions of the red ball for movement visualization
    speeds = [0]  # Starting with speed zero
    last_position = None  # Keep track of the last position
    last_position_white = None  # Keep track of the last white ball position
    movement_threshold = 2  # Threshold to determine movement, may need adjustment
    frame_counter = 0  # Counter for displaying "Hamle basladi"
    end_move_frame = int(13 * fps)  # Calculate the frame number at 13 seconds
    end_move_duration = int(5 * fps)  # Display "Hamle bitti" for 5 seconds

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        current_frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # Red ball detection and tracking
        lower_red = np.array([0, 170, 50])
        upper_red = np.array([10, 255, 255])
        mask1 = cv2.inRange(hsv_frame, lower_red, upper_red)
        lower_red = np.array([170, 150, 50])
        upper_red = np.array([180, 255, 255])
        mask2 = cv2.inRange(hsv_frame, lower_red, upper_red)
        red_mask = cv2.bitwise_or(mask1, mask2)
        red_contours, _ = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if red_contours:
            largest_red = max(red_contours, key=cv2.contourArea)
            (x, y), radius = cv2.minEnclosingCircle(largest_red)
            if radius > 5:
                positions.append((int(x), int(y)))
                if last_position is not None:
                    movement = np.sqrt((x - last_position[0])*2 + (y - last_position[1])*2)
                    if movement > movement_threshold:
                        speed = movement * fps  # Convert movement to speed (pixels per second)
                        speeds.append(speed)
                    else:
                        speeds.append(0)  # No significant movement, speed is zero
                cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 0), 2)  # Draw the red ball
                cv2.putText(frame, f"Speed: {speeds[-1]:.2f} px/s", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                cv2.putText(frame, f"Position: ({int(x)}, {int(y)})", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255,0), 2)
                last_position = (int(x), int(y))

        # White ball movement detection
        white_mask = cv2.inRange(hsv_frame, np.array([0, 0, 200]), np.array([180, 20, 255]))
        white_contours, _ = cv2.findContours(white_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if white_contours:
            largest_white = max(white_contours, key=cv2.contourArea)
            (wx, wy), wradius = cv2.minEnclosingCircle(largest_white)
            if wradius > 5:
                if last_position_white is not None:
                    distance = np.sqrt((wx - last_position_white[0])*2 + (wy - last_position_white[1])*2)
                    if distance > 10:
                        if frame_counter == 0:
                            frame_counter = 3 * fps
                last_position_white = (wx, wy)

        if frame_counter > 0:
            cv2.putText(frame, "Hamle basladi", (frame_width - 260, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (70, 255, 0), 2)
            frame_counter -= 1

        if current_frame_number >= end_move_frame and current_frame_number < end_move_frame + end_move_duration:
            cv2.putText(frame, "Hamle bitti", (frame_width - 260, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (70, 255,0), 2)

        out.write(frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print("Video processing complete. Output saved to:", output_path)
    
# Define input and output video paths
input_video_path = 'C:/Users/acer/Downloads/vid_2.avi'
output_video_path = 'C:/Users/acer/Downloads/outputvid2.avi'

track_and_annotate_ball(input_video_path, output_video_path)

C:\Users\acer\AppData\Local\Temp/ipykernel_12164/586159018.py:68: RuntimeWarning: invalid value encountered in sqrt
  distance = np.sqrt((wx - last_position_white[0])*2 + (wy - last_position_white[1])*2)
C:\Users\acer\AppData\Local\Temp/ipykernel_12164/586159018.py:49: RuntimeWarning: invalid value encountered in sqrt
  movement = np.sqrt((x - last_position[0])*2 + (y - last_position[1])*2)


Video processing complete. Output saved to: C:/Users/acer/Downloads/outputvid2.avi
